In [14]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime as dt
import csv
from datetime import datetime


def readCSVFile(fileName):
    list=[]
    with open('twitter_data.csv', 'rt') as f:
        reader = csv.DictReader(f)
        for row in reader:
            list.append({'full_text':str(row['full_text']),'user_name':str(row['user_name'])})            
    return list

def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, data)
        print('Message published successfully.\n Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    


In [15]:
if __name__ == '__main__':
   
    topic = 'twitter_data_analysis'
    cRows = readCSVFile(topic)
    
    print('Publishing records..')
    producer = connect_kafka_producer()
    start_index=0
    
    rows_to_send=1
#     late_rows_to_send=random.randint(1,3)
#     late_data_to_send = []
    
    while True:        
        
        ts = {'ts': int(dt.datetime.now().timestamp())}
        to_send = cRows[start_index:start_index+rows_to_send]
        #increase the start index by the number of rows taken
        start_index = start_index+rows_to_send
        #append the timestamp into the object to be sent
        data=[dict(item,**ts) for item in to_send]        
        
        #late data
#         to_send = cRows[start_index:start_index+late_rows_to_send]
        #increase the start index by the number of rows taken
#         start_index = start_index+late_rows_to_send
        #append the timestamp into the object to be sent        
#         late_data=[dict(item,**ts) for item in to_send]
            
        #publish both current and late data
        #the late data (from previous cycle is appended to the current data to be sent)
#         data.extend(late_data_to_send)
        publish_message(producer, topic, data)
        
#         #display the timestamps being sent
#         for d in data:
#             print(datetime.fromtimestamp(d['ts']),'impressions:',d['Impressions'])
#         print('-------------------------')
        
#         #after the current batch is sent, set the late data 
#         late_data_to_send=late_data
        
#         #reset to start from begining
#         if(start_index>=len(cRows)):
#             start_index=0
        sleep(5)
        

Publishing records..
Message published successfully.
 Data: [{'full_text': 'After the climate election: shellshocked green groups remain resolute https://t.co/wyJzmAcyiD', 'user_name': 'PIPELINEPETE', 'ts': 1616225517}]
Message published successfully.
 Data: [{'full_text': '@narendramodi @smritiirani Coverage of indian election on SBS tv channel, Australia. Jai hind 🇮🇳🙏 https://t.co/90qplBEAf8', 'user_name': 'Narinder Parmar', 'ts': 1616225522}]
Message published successfully.
 Data: [{'full_text': '@workmanalice Do you know if Facebook is releasing an election post-mortem in Australia? They looked into the midterms, but were we important enough to bother?', 'user_name': 'Peter Wells', 'ts': 1616225527}]
Message published successfully.
 Data: [{'full_text': '@vanbadham We all understand we have a compulsory preference system. Vote 1 mightn’t go to the major but 2 or 3 usually does.\nMajority of Australia wanted LNP, that’s the facts.\nThis is nothing like the USA System.', 'user_name':

KeyboardInterrupt: 